# Testing the API

In [1]:
import requests
import os
from IPython.display import Image

In [2]:
#most recent
url = 'https://epic.gsfc.nasa.gov/api/natural/'
#url by date
date = '20170604'
url = f'https://epic.gsfc.nasa.gov/api/natural/date/{date}'
res = requests.get(url)

In [3]:
# Example data retrieved
print(f'There are {len(res.json())} images available')
res.json()[1]

There are 21 images available


{'identifier': '20170604014201',
 'caption': "This image was taken by NASA's EPIC camera onboard the NOAA DSCOVR spacecraft",
 'image': 'epic_1b_20170604014201',
 'version': '03',
 'centroid_coordinates': {'lat': 25.36377, 'lon': 163.73291},
 'dscovr_j2000_position': {'x': 196247.999971,
  'y': 1321107.183262,
  'z': 634766.316667},
 'lunar_j2000_position': {'x': -386463.468503,
  'y': -91763.635722,
  'z': -12475.535012},
 'sun_j2000_position': {'x': 43331983.133362,
  'y': 133441654.266659,
  'z': 57847560.872444},
 'attitude_quaternions': {'q0': 0.51265,
  'q1': -0.32367,
  'q2': -0.67941,
  'q3': 0.41331},
 'date': '2017-06-04 01:37:13',
 'coords': {'centroid_coordinates': {'lat': 25.36377, 'lon': 163.73291},
  'dscovr_j2000_position': {'x': 196247.999971,
   'y': 1321107.183262,
   'z': 634766.316667},
  'lunar_j2000_position': {'x': -386463.468503,
   'y': -91763.635722,
   'z': -12475.535012},
  'sun_j2000_position': {'x': 43331983.133362,
   'y': 133441654.266659,
   'z': 57847

In [4]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
img = f"https://epic.gsfc.nasa.gov/archive/{collection['nat']}/{date[:4]}/{date[4:6]}/{date[6:8]}/{size['med']}/{res.json()[0]['image']}.{size['med']}"
print(img)
display(Image(url=img))

https://epic.gsfc.nasa.gov/archive/natural/2017/06/04/jpg/epic_1b_20170604003634.jpg


In [5]:
# Access the image
size = {'big':'.png', 'med':'jpg'}
collection = {'nat': 'natural', 'enh':'enhanced'}
for e in res.json():
    img = f"https://epic.gsfc.nasa.gov/archive/{collection['nat']}/{date[:4]}/{date[4:6]}/{date[6:8]}/jpg/{e['image']}.{size['med']}"
    #print(img)
    display(Image(url=img))

In [6]:
# Example data retrieved
print(res.json()[0]['identifier'])
YYYYMMDD = res.json()[0]['identifier']
year = YYYYMMDD[:4]
month = YYYYMMDD[4:6]
day = YYYYMMDD[6:8]
print(year, month, day)

20170604003634
2017 06 04


In [7]:
# Define when
def getSatelite(YYYYMMDD,queryParams=dict(),apiKey=""):
    
year = YYYYMMDD[:4]
month = YYYYMMDD[4:6]
day = YYYYMMDD[6:8]
    
    
    # Construct the resource url
    url = f'https://epic.gsfc.nasa.gov/api/natural'
    # metadata = f"https://epic.gsfc.nasa.gov/api/natural/date/{year}-{month}-{day}"
    
    # If apiKey is defined, pass a header
    headers = {"Authorization":f"token {apiKey}"} if apiKey else {}

    # Perform the request
    res = requests.get(url, params=queryParams, headers=headers)
    print(res.status_code, res.url)
    
    # Extract json from body response
    return res.json()


#specify requests url
metadata = f"https://epic.gsfc.nasa.gov/api/natural/date/{year}-{month}-{day}"

IndentationError: expected an indented block (<ipython-input-7-0b180c61e94e>, line 4)

# Changing the PHP api to Python code

In [ ]:
# Define when
month = '08'
day = '21'
year = '2017'

#specify requests url
metadata = f"https://epic.gsfc.nasa.gov/api/natural/date/{year}-{month}-{day}"

meta = file_get_contents($metadata);   # get the metadata for that date and collection
arr = json_decode($meta);  # decode the metadata

foreach($arr as $item) {
    $name = $item->image . '.png';
    $archive = "https://epic.gsfc.nasa.gov/archive/natural/{$year}/{$month}/{$day}/png/";

    $source = $archive . $name;
    $destination = '/path/to/downloads/' . $name;

    copy($source, $destination);    // download and copy the image
    }

In [ ]:
def getSatelite(date, queryParams=dict(),apiKey=""):
    
    # Construct the resource url
    url = f'https://epic.gsfc.nasa.gov/api/natural/{date}'
    
    # If apiKey is defined, pass a header
    headers = {"Authorization":f"token {apiKey}"} if apiKey else {}

    # Perform the request
    res = requests.get(url, params=queryParams, headers=headers)
    print(res.status_code, res.url)
    
    # Extract json from body response
    return res.json()

# Working with datetime

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
df = pd.DataFrame('INPUT/volcanic-explosions.csv')

In [ ]:
date_rng = pd.date_range(start='1/1/2015', end='10/04/2020', freq='D')
date_rng

In [ ]:
df = pd.DataFrame(date_rng, columns=['date'])
df['data'] = np.random.randint(0,100, size=(len(date_rng)))
df.head()

In [ ]:
df['datetime'] = pd.to_datetime(df.date)
df = df.set_index('datetime')

#list only the second day of each month
#df[df.index.day == 2]

# calculate mean of each year
df.resample('M').mean()

In [ ]:
year= 2020
month= 2
day=6

datetime.date(year,month,day).weekday()

In [ ]:
datetime.date.today()